In [155]:
import pandas as pd
import re

dataframe = pd.read_csv("wggesucht_scraped_24_05.csv")

display(dataframe)

,Url,Title,Size,PriceTotal,Price,ExtraCosts,OtherCosts,Deposit,RedemptionAgreement,DateFree,...,OnlineSince,Adress,Details,LookingFor,AmountofIcons,AmountofIconsNeeded,DescriptionRoom,DescriptionLocation,DescriptionLife,description_other
0,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Room in a WG of 4 close to Freimann U6,12m²,670€,670€,n.a.,n.a.,n.a.,n.a.,01.07.2024,...,32 Minuten,Freisinger Landstr. 18\n ...,12m²\n \n ...,Geschlecht egal\n ...,3,n.a.,"The room is furnished with desk, bed and wardr...",Very close to the Freimann stop (U6). Buses 18...,You will be sharing the apartment with me (Ita...,"The kitchen, which is a separate room in the a..."
1,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Beste Lage - Zimmer frei bei Studentenverbindung,20m²,350€,350€,n.a.,0€,600€,0€,24.05.2024,...,6 Minuten,Königinstr. 20\n ...,20m²\n \n ...,Mann\n ...,12,n.a.,Du wirst demnächst Dein Studium an der LMU/TU ...,Top Lage direkt am Englischen Garten - Zweimal...,"Wir sind ein ""Corps"", eine Art von Studentenv...","Da wir besonderen Wert darauf legen, dass wir ..."
2,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP LAGE - U3 Nahe Olympia-Einkaufszentrum,11m²,790€,740€,50€,0€,2200€,0€,24.05.2024,...,37 Minuten,Seydlitzstraße 63\n ...,11m²\n \n ...,Geschlecht egal\n,8,4,Gemütliches WG Zimmer voll möbliert mit Bett m...,"Sehr gute Lage, Bus-und U-Bahn-Haltestellen na...",Gemischte Wohngemeinschaft mit Studenten und B...,Verfügbar ab sofort.
3,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP UNI LOCATION near U3/U6: Münchner Freiheit,14m²,970€,911€,59€,0€,2673€,0€,01.08.2024,...,37 Minuten,Germaniastraße 3\n ...,14m²\n \n ...,Geschlecht egal\n,7,3,"Schlafzimmer ist voll Möbliert (Bett, Schrank,...",Die Wohnung befindet sich in einer spitzen Lag...,4 junge Männer Anfang 20 Jahre. \n\r\n Gemeins...,Bitte kontaktieren Sie uns über WG-Gesucht. \r...
4,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,+++ WG-Zimmer in renovierter 4er WG verfügbar!...,14m²,800€,715€,85€,0€,2145€,0€,01.07.2024,...,39 Minuten,Schlierseestraße 14\n ...,14m²\n \n ...,Geschlecht egal\n,12,5,Wunderschönes Zimmer in 4er WG ab dem 01.07.20...,Die Schlierseestraße 14 liegt in einer attrakt...,Willkommen in unserer charmanten WG in München...,Link zum Wohnungs-Bewerbungsformular (Aplicati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Wohngemeinschaft in München,108m²,1220€,1220€,n.a.,n.a.,1830€,n.a.,05.08.2024,...,17.05.2024,Tumblingerstraße Glockenbachviertel 17\n ...,108m²\n \n ...,Geschlecht egal\n,3,n.a.,Wohnen Sie nur wenige Schritte von der Isar en...,munich,n.a.,**Bitte beachten: PAARE** sind nur in Schlafzi...
1131,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,n.a.,1325€,1325€,n.a.,n.a.,1000€,n.a.,24.05.2024,...,17.05.2024,Heßstraße 74\n ...,Zimmer in 2er WG\nWohnungsgröße: 1m²\n2er WG\n...,Geschlecht egal\n,4,n.a.,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,munich,n.a.,**Dieser Vermieter akzeptiert keine Studenten!...
1132,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Privatzimmer in Laim...,12m²,1020€,1020€,n.a.,n.a.,n.a.,n.a.,19.08.2024,...,1 Tag,Tübinger Straße 3c\n ...,12m²\n \n ...,Geschlecht egal\n,4,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-896965
1133,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Private Room in Unte...,14m²,960€,960€,n.a.,n.a.,n.a.,n.a.,05.08.2024,...,1 Tag,Grünwalderstraße 119\n ...,14m²\n \n ...,Geschlecht egal\n,4,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-380836


Shape the data to be able to do a regression analysis

1. Idea: If the length of a title is too short, it might not be as appealing or the advert isn't made well hence it's negatively correlated

In [156]:
dataframe['Titel_length'] = dataframe['Title'].str.len()

2. Idea: The size of the WG is highly correlated with the pricing.

In [157]:
dataframe['Size'] = dataframe['Size'].str.replace(r'\D+', '', regex=True)

3. Idea: Relevant is only the toal price and the deposit

In [158]:
# Remove columns
dataframe = dataframe.drop(['Price', 'ExtraCosts', 'OtherCosts'], axis=1)

columns_to_replace = ['RedemptionAgreement', 'DateFreeTill']
dataframe[columns_to_replace] = dataframe[columns_to_replace].replace('n.a.', 0)

# Remove '€' symbol from 'PriceTotal' and 'Deposit' columns
dataframe['PriceTotal'] = dataframe['PriceTotal'].str.replace('€', '')
dataframe['Deposit'] = dataframe['Deposit'].str.replace('€', '')


4. Idea: Transform the column onlineSince to hours

In [159]:
dataframe['OnlineSince'] = dataframe['OnlineSince'].astype(str)

# Function to transform the values in the "OnlineSince" column
def transform_online_since(row):
    if 'Minuten' in row:
        return 1
    elif 'Stunde' in row:
        return 1
    elif 'Stunden' in row:
        return int(''.join(filter(str.isdigit, row)))
    elif 'Tag' in row:
        return 24
    else:
        refering_date = pd.to_datetime('24.05.2024', format='%d.%m.%Y')
        date = pd.to_datetime(row, format='%d.%m.%Y')
        hours_difference = (refering_date - date).total_seconds() / 3600
        return hours_difference

# Apply the transformation function to the "OnlineSince" column
dataframe['OnlineSince'] = dataframe['OnlineSince'].apply(transform_online_since)

display(dataframe)

,Url,Title,Size,PriceTotal,Deposit,RedemptionAgreement,DateFree,DateFreeTill,OnlineSince,Adress,Details,LookingFor,AmountofIcons,AmountofIconsNeeded,DescriptionRoom,DescriptionLocation,DescriptionLife,description_other,Titel_length
0,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Room in a WG of 4 close to Freimann U6,12,670,n.a.,0,01.07.2024,0,1.0,Freisinger Landstr. 18\n ...,12m²\n \n ...,Geschlecht egal\n ...,3,n.a.,"The room is furnished with desk, bed and wardr...",Very close to the Freimann stop (U6). Buses 18...,You will be sharing the apartment with me (Ita...,"The kitchen, which is a separate room in the a...",38
1,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Beste Lage - Zimmer frei bei Studentenverbindung,20,350,600,0€,24.05.2024,0,1.0,Königinstr. 20\n ...,20m²\n \n ...,Mann\n ...,12,n.a.,Du wirst demnächst Dein Studium an der LMU/TU ...,Top Lage direkt am Englischen Garten - Zweimal...,"Wir sind ein ""Corps"", eine Art von Studentenv...","Da wir besonderen Wert darauf legen, dass wir ...",48
2,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP LAGE - U3 Nahe Olympia-Einkaufszentrum,11,790,2200,0€,24.05.2024,0,1.0,Seydlitzstraße 63\n ...,11m²\n \n ...,Geschlecht egal\n,8,4,Gemütliches WG Zimmer voll möbliert mit Bett m...,"Sehr gute Lage, Bus-und U-Bahn-Haltestellen na...",Gemischte Wohngemeinschaft mit Studenten und B...,Verfügbar ab sofort.,42
3,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP UNI LOCATION near U3/U6: Münchner Freiheit,14,970,2673,0€,01.08.2024,0,1.0,Germaniastraße 3\n ...,14m²\n \n ...,Geschlecht egal\n,7,3,"Schlafzimmer ist voll Möbliert (Bett, Schrank,...",Die Wohnung befindet sich in einer spitzen Lag...,4 junge Männer Anfang 20 Jahre. \n\r\n Gemeins...,Bitte kontaktieren Sie uns über WG-Gesucht. \r...,46
4,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,+++ WG-Zimmer in renovierter 4er WG verfügbar!...,14,800,2145,0€,01.07.2024,0,1.0,Schlierseestraße 14\n ...,14m²\n \n ...,Geschlecht egal\n,12,5,Wunderschönes Zimmer in 4er WG ab dem 01.07.20...,Die Schlierseestraße 14 liegt in einer attrakt...,Willkommen in unserer charmanten WG in München...,Link zum Wohnungs-Bewerbungsformular (Aplicati...,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Wohngemeinschaft in München,108,1220,1830,0,05.08.2024,0,168.0,Tumblingerstraße Glockenbachviertel 17\n ...,108m²\n \n ...,Geschlecht egal\n,3,n.a.,Wohnen Sie nur wenige Schritte von der Isar en...,munich,n.a.,**Bitte beachten: PAARE** sind nur in Schlafzi...,27
1131,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,,1325,1000,0,24.05.2024,0,168.0,Heßstraße 74\n ...,Zimmer in 2er WG\nWohnungsgröße: 1m²\n2er WG\n...,Geschlecht egal\n,4,n.a.,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,munich,n.a.,**Dieser Vermieter akzeptiert keine Studenten!...,67
1132,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Privatzimmer in Laim...,12,1020,n.a.,0,19.08.2024,31.03.2025,24.0,Tübinger Straße 3c\n ...,12m²\n \n ...,Geschlecht egal\n,4,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-896965,55
1133,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Private Room in Unte...,14,960,n.a.,0,05.08.2024,31.03.2025,24.0,Grünwalderstraße 119\n ...,14m²\n \n ...,Geschlecht egal\n,4,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-380836,73


5. Idea: Geografische Lage der PLZ

In [160]:
postal_codes_frame = pd.read_csv('Munich_Postal_Codes.csv')

postal_codes = []

for address in dataframe['Adress']:
    # Extract the 5-digit postal code using regular expressions
    postal_code = re.findall(r'\b\d{5}\b', address)
    
    if postal_code:
        postal_codes.append(postal_code[0])
    else:
        postal_codes.append(None)

# Replace the "Adress" column with the extracted postal codes
dataframe['Adress'] = postal_codes

# Loop over the 'address' column in the 'dataframe'
for index, row in dataframe.iterrows():
    address = row['Adress']

    if address is None:
        dataframe.at[index, 'Adress'] = 0
        continue

    for index2, row in postal_codes_frame.iterrows():
                    
        if address in row['Postal Codes']:
            dataframe.at[index, 'Adress'] = row['Category']
        else:
            continue
display(dataframe)

,Url,Title,Size,PriceTotal,Deposit,RedemptionAgreement,DateFree,DateFreeTill,OnlineSince,Adress,Details,LookingFor,AmountofIcons,AmountofIconsNeeded,DescriptionRoom,DescriptionLocation,DescriptionLife,description_other,Titel_length
0,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Room in a WG of 4 close to Freimann U6,12,670,n.a.,0,01.07.2024,0,1.0,3,12m²\n \n ...,Geschlecht egal\n ...,3,n.a.,"The room is furnished with desk, bed and wardr...",Very close to the Freimann stop (U6). Buses 18...,You will be sharing the apartment with me (Ita...,"The kitchen, which is a separate room in the a...",38
1,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Beste Lage - Zimmer frei bei Studentenverbindung,20,350,600,0€,24.05.2024,0,1.0,4,20m²\n \n ...,Mann\n ...,12,n.a.,Du wirst demnächst Dein Studium an der LMU/TU ...,Top Lage direkt am Englischen Garten - Zweimal...,"Wir sind ein ""Corps"", eine Art von Studentenv...","Da wir besonderen Wert darauf legen, dass wir ...",48
2,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP LAGE - U3 Nahe Olympia-Einkaufszentrum,11,790,2200,0€,24.05.2024,0,1.0,3,11m²\n \n ...,Geschlecht egal\n,8,4,Gemütliches WG Zimmer voll möbliert mit Bett m...,"Sehr gute Lage, Bus-und U-Bahn-Haltestellen na...",Gemischte Wohngemeinschaft mit Studenten und B...,Verfügbar ab sofort.,42
3,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP UNI LOCATION near U3/U6: Münchner Freiheit,14,970,2673,0€,01.08.2024,0,1.0,3,14m²\n \n ...,Geschlecht egal\n,7,3,"Schlafzimmer ist voll Möbliert (Bett, Schrank,...",Die Wohnung befindet sich in einer spitzen Lag...,4 junge Männer Anfang 20 Jahre. \n\r\n Gemeins...,Bitte kontaktieren Sie uns über WG-Gesucht. \r...,46
4,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,+++ WG-Zimmer in renovierter 4er WG verfügbar!...,14,800,2145,0€,01.07.2024,0,1.0,3,14m²\n \n ...,Geschlecht egal\n,12,5,Wunderschönes Zimmer in 4er WG ab dem 01.07.20...,Die Schlierseestraße 14 liegt in einer attrakt...,Willkommen in unserer charmanten WG in München...,Link zum Wohnungs-Bewerbungsformular (Aplicati...,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Wohngemeinschaft in München,108,1220,1830,0,05.08.2024,0,168.0,2,108m²\n \n ...,Geschlecht egal\n,3,n.a.,Wohnen Sie nur wenige Schritte von der Isar en...,munich,n.a.,**Bitte beachten: PAARE** sind nur in Schlafzi...,27
1131,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,,1325,1000,0,24.05.2024,0,168.0,4,Zimmer in 2er WG\nWohnungsgröße: 1m²\n2er WG\n...,Geschlecht egal\n,4,n.a.,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,munich,n.a.,**Dieser Vermieter akzeptiert keine Studenten!...,67
1132,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Privatzimmer in Laim...,12,1020,n.a.,0,19.08.2024,31.03.2025,24.0,2,12m²\n \n ...,Geschlecht egal\n,4,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-896965,55
1133,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Private Room in Unte...,14,960,n.a.,0,05.08.2024,31.03.2025,24.0,4,14m²\n \n ...,Geschlecht egal\n,4,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-380836,73


6. Idea: Sentiment Analysis of text fields:

In [161]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Perform sentiment analysis on the columns
details_sentiment = dataframe['Details'].apply(lambda x: sia.polarity_scores(x)['compound'])
description_room_sentiment = dataframe['DescriptionRoom'].apply(lambda x: sia.polarity_scores(x)['compound'])
description_location_sentiment = dataframe['DescriptionLocation'].apply(lambda x: sia.polarity_scores(x)['compound'])
description_life_sentiment = dataframe['DescriptionLife'].apply(lambda x: sia.polarity_scores(x)['compound'])
description_other_sentiment = dataframe['description_other'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Save the sentiment analysis results
dataframe['analysis_det'] = details_sentiment
dataframe['analysis_desr'] = description_room_sentiment
dataframe['analysis_desl'] = description_location_sentiment
dataframe['analysis_desl'] = description_life_sentiment
dataframe['analysis_deso'] = description_other_sentiment

display(dataframe)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Gero\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Url,Title,Size,PriceTotal,Deposit,RedemptionAgreement,DateFree,DateFreeTill,OnlineSince,Adress,...,AmountofIconsNeeded,DescriptionRoom,DescriptionLocation,DescriptionLife,description_other,Titel_length,analysis_det,analysis_desr,analysis_desl,analysis_deso
0,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Room in a WG of 4 close to Freimann U6,12,670,n.a.,0,01.07.2024,0,1.0,3,...,n.a.,"The room is furnished with desk, bed and wardr...",Very close to the Freimann stop (U6). Buses 18...,You will be sharing the apartment with me (Ita...,"The kitchen, which is a separate room in the a...",38,0.4588,0.0000,0.8625,0.0772
1,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Beste Lage - Zimmer frei bei Studentenverbindung,20,350,600,0€,24.05.2024,0,1.0,4,...,n.a.,Du wirst demnächst Dein Studium an der LMU/TU ...,Top Lage direkt am Englischen Garten - Zweimal...,"Wir sind ein ""Corps"", eine Art von Studentenv...","Da wir besonderen Wert darauf legen, dass wir ...",48,0.0000,0.0000,-0.9020,-0.5994
2,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP LAGE - U3 Nahe Olympia-Einkaufszentrum,11,790,2200,0€,24.05.2024,0,1.0,3,...,4,Gemütliches WG Zimmer voll möbliert mit Bett m...,"Sehr gute Lage, Bus-und U-Bahn-Haltestellen na...",Gemischte Wohngemeinschaft mit Studenten und B...,Verfügbar ab sofort.,42,0.0000,0.8122,0.3400,0.0000
3,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,TOP UNI LOCATION near U3/U6: Münchner Freiheit,14,970,2673,0€,01.08.2024,0,1.0,3,...,3,"Schlafzimmer ist voll Möbliert (Bett, Schrank,...",Die Wohnung befindet sich in einer spitzen Lag...,4 junge Männer Anfang 20 Jahre. \n\r\n Gemeins...,Bitte kontaktieren Sie uns über WG-Gesucht. \r...,46,0.0000,-0.9097,0.0000,0.0000
4,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,+++ WG-Zimmer in renovierter 4er WG verfügbar!...,14,800,2145,0€,01.07.2024,0,1.0,3,...,5,Wunderschönes Zimmer in 4er WG ab dem 01.07.20...,Die Schlierseestraße 14 liegt in einer attrakt...,Willkommen in unserer charmanten WG in München...,Link zum Wohnungs-Bewerbungsformular (Aplicati...,52,0.0000,-0.5255,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Wohngemeinschaft in München,108,1220,1830,0,05.08.2024,0,168.0,2,...,n.a.,Wohnen Sie nur wenige Schritte von der Isar en...,munich,n.a.,**Bitte beachten: PAARE** sind nur in Schlafzi...,27,0.0000,-0.6360,0.0000,-0.9545
1131,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,,1325,1000,0,24.05.2024,0,168.0,4,...,n.a.,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,munich,n.a.,**Dieser Vermieter akzeptiert keine Studenten!...,67,0.0000,0.0000,0.0000,-0.9783
1132,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Privatzimmer in Laim...,12,1020,n.a.,0,19.08.2024,31.03.2025,24.0,2,...,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-896965,55,0.0000,-0.9661,0.0000,0.0000
1133,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Private Room in Unte...,14,960,n.a.,0,05.08.2024,31.03.2025,24.0,4,...,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: UBK-380836,73,0.0000,-0.9828,0.0000,0.0000


7. Idea: Drop all old or not relevant columns

In [162]:
columns_to_drop = ['Url', 'Title', 'RedemptionAgreement', 'DateFreeTill', 'DescriptionRoom', 'DescriptionLocation', 'DescriptionLife', 'description_other', 'Details','LookingFor', 'DateFree']
dataframe = dataframe.drop(columns=columns_to_drop)

dataframe.dropna(inplace=True)



Minor adjustments.

In [164]:
dataframe = dataframe.replace('[A-Za-z]', 0, regex=True).replace('', 0, regex=True)
display(dataframe)

dataframe.to_csv('wggesucht_scraped_24_05_cleaned.csv', index=False)

,Size,PriceTotal,Deposit,OnlineSince,Adress,AmountofIcons,AmountofIconsNeeded,Titel_length,analysis_det,analysis_desr,analysis_desl,analysis_deso
0,12,670,0,1.0,3,3,0,38,0.4588,0.0000,0.8625,0.0772
1,20,350,600,1.0,4,12,0,48,0.0000,0.0000,-0.9020,-0.5994
2,11,790,2200,1.0,3,8,4,42,0.0000,0.8122,0.3400,0.0000
3,14,970,2673,1.0,3,7,3,46,0.0000,-0.9097,0.0000,0.0000
4,14,800,2145,1.0,3,12,5,52,0.0000,-0.5255,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
1130,108,1220,1830,168.0,2,3,0,27,0.0000,-0.6360,0.0000,-0.9545
1131,0,1325,1000,168.0,4,4,0,67,0.0000,0.0000,0.0000,-0.9783
1132,12,1020,0,24.0,2,4,0,55,0.0000,-0.9661,0.0000,0.0000
1133,14,960,0,24.0,4,4,0,73,0.0000,-0.9828,0.0000,0.0000
